<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
Agents
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>

def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'SERPAPI_API_KEY': 'SERPAPI_API_KEY',
        'WEATHER_API_KEY': 'WEATHER_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        'GOOGLE_API_KEY': 'GOOGLE_API_KEY',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
WEATHER_API_KEY = all_keys['WEATHER_API_KEY']


# **1 <font color='orange'>|</font> Grundlagen von KI-Agenten**
---

<p><font color='black' size="5">
Was sind KI-Agenten?
</font></p>

**Definition und Abgrenzung zu anderen KI-Systemen:**    
KI-Agenten sind autonome Systeme, die ihre Umgebung wahrnehmen, Entscheidungen treffen und Aktionen ausführen können, um bestimmte Ziele zu erreichen. Im Gegensatz zu einfachen KI-Modellen, die nur auf Eingabe reagieren und Ausgabe produzieren, können Agenten proaktiv handeln und aus ihren Erfahrungen lernen.


**Autonomie, Reaktivität und zielgerichtetes Handeln:**    
Ein KI-Agent kann selbstständig (autonom) Entscheidungen treffen, auf Veränderungen in seiner Umgebung reagieren (Reaktivität) und hat ein oder mehrere Ziele, die sein Handeln leiten. Dabei verfolgt er langfristige Strategien statt nur auf unmittelbare Reize zu reagieren.


**Unterschied zwischen einfachen LLM-Anwendungen und Agenten:**    
Einfache LLM-Anwendungen (Large Language Models) wie ein Chatbot antworten lediglich auf Anfragen, während Agenten darüber hinaus eigene Entscheidungen treffen, Tools nutzen und zielgerichtet handeln können. Ein Agent kann beispielsweise entscheiden, wann er eine Suchanfrage starten sollte, um eine Frage besser zu beantworten.


<p><font color='black' size="5">
Ein erster Agent mit LangChain
</font></p>

+ **Setup der Entwicklungsumgebung:** Installation und Konfiguration der notwendigen Bibliotheken, insbesondere LangChain und die Anbindung an ein LLM wie OpenAI.
+ **ToolSet definieren:** Definition der druch die Agnets benutzen Tools.
+ **Implementierung eines einfachen Agenten:** Ein Beispiel für einen einfachen Agenten, der Fragen beantwortet und Berechnungen durchführt:


**Setup der Entwicklungsumgebung:**

In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system -q -U langchain-openai langchain-community google-search-results wikipedia

In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system -q -U langchain-openai langchain-community google-search-results wikipedia

**Zentrale Tool-Bibliothek**

In [ ]:
import os
import re
import requests
import json
from datetime import datetime
from langchain_core.tools import Tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Tool-Funktionen definieren
def get_weather(location):
    """Gibt das aktuelle Wetter für einen bestimmten Ort zurück."""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return f"Aktuelle Temperatur in {location}: {data['main']['temp']}°C, {data['weather'][0]['description']}"

def read_file(file_path):
    """Liest den Inhalt einer Datei."""
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    return f"Fehler: Datei {file_path} nicht gefunden."

def write_file(args):
    """Schreibt Inhalt in eine Datei. Format: 'pfad//inhalt'"""
    try:
        file_path, content = args.split("//", 1)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return f"Erfolgreich in {file_path} geschrieben."
    except Exception as e:
        return f"Fehler beim Schreiben: {str(e)}"

def days_between_dates(date_string):
    """Berechnet Tage zwischen zwei Datumsangaben im Format DD.MM.YYYY"""
    try:
        date1_str, date2_str = date_string.split(",")
        date1_str = date1_str.strip()
        date2_str = date2_str.strip()
        date1 = datetime.strptime(date1_str, "%d.%m.%Y").date()
        date2 = datetime.strptime(date2_str, "%d.%m.%Y").date()
        return str((date2 - date1).days)
    except ValueError:
        return "Ungültiges Datumsformat. Bitte verwende DD.MM.YYYY."

def extract_math_expression(text):
    """Extrahiert mathematischen Ausdruck aus Text."""
    match = re.search(r'([\d+\-*/().]+)', text)
    return match.group(0) if match else None

def calculate(expression):
    """Führt mathematische Berechnungen durch."""
    try:
        math_expr = extract_math_expression(expression)
        if math_expr:
            return str(eval(math_expr))
        return "Keine gültige Berechnung gefunden."
    except:
        return "Fehler bei der Berechnung."

# Externe APIs
serpapi = SerpAPIWrapper()
wiki = WikipediaAPIWrapper()

# Tools-Liste, die in allen Beispielen verwendet werden kann
tools = [
    Tool(name="search", func=serpapi.run, description="Infos aus dem Internet"),
    Tool(name="wiki", func=lambda x: wiki.run(x).split("\n")[0] if wiki.run(x) else "Keine relevanten Infos gefunden.",
         description="Fakten aus Wikipedia"),
    Tool(name="calculator", func=calculate, description="Mathematische Berechnungen"),
    Tool(name="weather", func=get_weather, description="Gibt das aktuelle Wetter für einen bestimmten Ort zurück"),
    Tool(name="read_file", func=read_file, description="Liest den Inhalt einer Datei. Input sollte der Dateipfad sein."),
    Tool(name="write_file", func=write_file, description="Überschreibt eine Datei mit neuem Inhalt. Input sollte im Format 'pfad//inhalt' sein."),
    Tool(name="days_between_dates", func=days_between_dates, description="Berechnet die Anzahl der Tage zwischen zwei Datumsangaben im Format DD.MM.YYYY.")
]

**Implementierung eines einfachen Agenten:**

Erläuterung der einzelnen Funktionen:

* `ChatOpenAI` ermöglicht die Nutzung von OpenAI's Chat-Modellen.
* `create_tool_calling_agent` und `AgentExecutor` erschaffen und führen einen Agenten aus, der Werkzeuge nutzen kann.
* `ChatPromptTemplate` und `MessagesPlaceholder` definieren die Struktur für Nachrichten und Vorlagen im Chat.
* `IPython.display display, Markdown` ermöglichen die Darstellung von formatiertem Text in einer Jupyter-Umgebung.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# Prompt erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent mit Zugang zu Tools. Bitte Formeln in $ Formel $ formatieren."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Agent erstellen und ausführen
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True
)

# Anfrage stellen
response = agent_executor.invoke({
    "input": "Wie viel ist 12 * 12? Und wer ist Taylor Swift?",
    "chat_history": []
})

# Ergebnis anzeigen
display(Markdown("# 📝 Antwort des Agenten"))
display(Markdown(response["output"]))

# Ausgabe:
# Das Ergebnis von $12 \times 12$ ist $144$.
#
# ## Wer ist Taylor Swift?
# Taylor Alison Swift (geboren am 13. Dezember 1989) ist eine amerikanische Singer-Songwriterin,
# die für ihr autobiografisches Songwriting, ihre künstlerische Vielseitigkeit und
# ihren kulturellen Einfluss bekannt ist...

Der `agent_scratchpad` ist ein entscheidender Bestandteil des Agenten-Frameworks, insbesondere bei der Verwendung von Werkzeugen (Tools):

* **Arbeitsbereich für den Agenten:**
    * Der `agent_scratchpad` dient als temporärer Arbeitsbereich, in dem der Agent seine Überlegungen, Aktionen und Beobachtungen während der Ausführung speichert.
    * Es ist der Ort, an dem der Agent seine "Denkschritte" festhält, bevor er eine endgültige Antwort gibt.
* **Kommunikation mit dem Sprachmodell:**
    * Wenn der Agent ein Werkzeug aufruft, werden die Details des Aufrufs (z. B. der Name des Werkzeugs und die Eingabeparameter) in den `agent_scratchpad` geschrieben.
    * Nachdem das Werkzeug ausgeführt wurde, wird das Ergebnis (die "Beobachtung") ebenfalls in den `agent_scratchpad` geschrieben.
    * Das Sprachmodell verwendet den Inhalt des `agent_scratchpad`, um zu entscheiden, welche nächsten Schritte es unternehmen soll (z. B. ein weiteres Werkzeug aufrufen oder eine endgültige Antwort geben).
* **Debugging und Transparenz:**
    * Der `agent_scratchpad` kann auch für Debugging-Zwecke nützlich sein, da er Einblicke in die Denkprozesse des Agenten bietet.
    * Es macht die Aktionskette des Agenten transparenter.

Im Kontext des Prompts:

* `MessagesPlaceholder(variable_name="agent_scratchpad")` fügt den Inhalt des `agent_scratchpad` in den Prompt ein, der an das Sprachmodell gesendet wird.
* Dadurch kann das Sprachmodell die vorherigen Aktionen und Beobachtungen des Agenten berücksichtigen, wenn es seine nächste Antwort generiert.

Zusammengefasst ist der `agent_scratchpad` ein dynamischer Bereich, der die Kommunikation und das Gedächtnis des Agenten während der Ausführung von Werkzeugen ermöglicht.


# **2 <font color='orange'>|</font> Grundlegende Agentenarchitekturen**
---

In diesem Abschnitt werden vier grundlegenden Agentenarchitekturen vorgestellt, die als Basis für komplexere Anwendungen dienen. Jede Architektur folgt einem eigenen Prinzip für Entscheidungsfindung und Handlung.

<p><font color='black' size="5">
Komponenten von Agentenarchitekturen
</font></p>


Die meisten modernen KI-Agentenarchitekturen bestehen aus mehreren Kernkomponenten:

**1. Reasoning-Komponente**

+ Verarbeitet Informationen und trifft Entscheidungen
+ Basiert typischerweise auf einem LLM
+ Kann explizite Gedankengänge (Chain-of-Thought) generieren

**2. Aktionskomponente**

+ Führt Aktionen in der Umgebung aus
+ Kann Tools aufrufen, APIs nutzen oder Systemaktionen auslösen
+ Übersetzt die Entscheidungen in konkrete Handlungen

**3. Beobachtungskomponente**

+ Nimmt Feedback aus der Umgebung wahr
+ Verarbeitet Ergebnisse vorheriger Aktionen
+ Liefert neue Informationen für den nächsten Reasoning-Schritt

**4. Planungskomponente**

+ Entwickelt Strategien für mehrschrittige Aufgaben
+ Zerlegt komplexe Ziele in Teilaufgaben
+ Kann kurz- und langfristige Pläne erstellen

**5. Gedächtniskomponente**

+ Speichert relevante Informationen
+ Behält den Kontext über mehrere Schritte hinweg
+ Ermöglicht Lernen aus vergangenen Erfahrungen



## 2.1 ReAct

**Reasoning and Acting**



**Kernprinzip:** Zyklischer Ablauf von Denken, Handeln und Beobachten.

**Charakteristika:**

- Explizite Gedankenformulierung vor jeder Aktion
- Beobachtung der Ergebnisse nach jeder Aktion
- Iterative Anpassung basierend auf Beobachtungen

**Reine ReAct-Implementierung:**

In [ ]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from datetime import datetime
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Spezifischer ReAct-Prompt aus LangChain Hub
prompt = hub.pull("hwchase17/react")

# Bei der Erstellung des Agenten müssen wir sicherstellen, dass die Tools korrekt definiert sind
agent = create_react_agent(llm, tools, prompt)

# Agent Executor zur Ausführung verwenden
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,  # Begrenzung der maximalen Iterationen
    early_stopping_method="force"  # Erzwingen eines Stopps bei Erreichen der max_iterations
)

# Aktuelles Datum für die Anfrage formatieren
aktuelles_datum_str = datetime.now().strftime("%d.%m.%Y")

# Typisches ReAct-Beispiel
query = f"Wann war die Fußball-WM 2022 und wie viele Tage ist das her? Heute ist der {aktuelles_datum_str}."
result = agent_executor.invoke({"input": query})

# Ergebnis anzeigen
display(Markdown("## 🏆 ReAct Agent Ergebnis"))
display(Markdown(result["output"]))

# Beispielausgabe:
# Die Fußball-WM 2022 fand vom 20. November 2022 bis zum 18. Dezember 2022 statt,
# und es sind 815 Tage seitdem vergangen.

## 2.2 MRKL

**Modular Reasoning, Knowledge and Language**



**Kernprinzip:** Kombination eines zentralen Sprachmodells mit spezialisierten Modulen (Tools).

**Charakteristika:**

- Modularität: Verschiedene Tools für spezialisierte Aufgaben
- Reasoning: Das Sprachmodell entscheidet, welches Tool zu verwenden ist
- Routing: Weiterleitung von Teilaufgaben an die passenden Tools
- Integration: Zusammenführung der Ergebnisse in eine kohärente Antwort

**Reine MRKL-Implementierung:**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Prompt von LangChain Hub laden
prompt = hub.pull("hwchase17/react")

# MRKL-Agent mit klar spezialisierten Modulen
agent = create_react_agent(llm, tools, prompt)
mrkl_agent = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Typisches MRKL-Beispiel mit verschiedenen Modulen
user_request = """
Wie ist das Wetter in Berlin?
Gib mir eine kurze Zusammenfassung zur Geschichte Berlins und
berechne die Tage zwischen dem 01.01.2023 und dem 31.12.2023.
"""

# Agent ausführen
response = mrkl_agent.invoke({"input": user_request})

# Ergebnis anzeigen
display(Markdown("# 🧩 MRKL Agent Ergebnis"))
display(Markdown(response["output"]))

# Beispielausgabe:
# Das Wetter in Berlin ist aktuell 8.45°C mit gebrochenen Wolken.
# Berlin hat eine reiche und komplexe Geschichte, die bis ins 13. Jahrhundert zurückreicht
# und sich über Kriege, Teilungen und Wiedervereinigungen erstreckt.
# Die Anzahl der Tage zwischen dem 01.01.2023 und dem 31.12.2023 beträgt 364 Tage.

## 2.3 Reflective Agents



**Kernprinzip:** Selbstbewertung und -verbesserung durch kritische Analyse der eigenen Outputs.

**Charakteristika:**

- Self-criticism: Kritische Bewertung der eigenen Ausgaben
- Verbesserungszyklen: Mehrfache Iteration zur Qualitätsverbesserung
- Meta-Kognition: "Denken über das Denken"

**Reine Reflective Agent-Implementierung:**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import display, Markdown

# LLMs mit unterschiedlichen Einstellungen
answer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # Kreativere Erstantwort
reflection_llm = ChatOpenAI(model="gpt-4o", temperature=0)      # Kritische Bewertung

# Prompts für die drei Phasen des Reflection-Prozesses
answer_prompt = PromptTemplate.from_template("Beantworte: {question}")
reflection_prompt = PromptTemplate.from_template("""
Bewerte:
Frage: {question}
Antwort: {answer}

Fehler/Verbesserungen?""")
improvement_prompt = PromptTemplate.from_template("""
Frage: {question}
Antwort: {answer}
Reflexion: {reflection}

Verbesserte Antwort:""")

# LCEL-Chains für die drei Phasen
answer_chain = answer_prompt | answer_llm | StrOutputParser()
reflection_chain = {
    "question": lambda x: x["question"],
    "answer": lambda x: x["answer"]
} | reflection_prompt | reflection_llm | StrOutputParser()
improvement_chain = {
    "question": lambda x: x["question"],
    "answer": lambda x: x["initial_answer"],
    "reflection": lambda x: x["reflection"]
} | improvement_prompt | answer_llm | StrOutputParser()

# Ausführung des Reflective Agent
question = "Hauptursachen des Klimawandels?"
initial_answer = answer_chain.invoke({"question": question})
reflection = reflection_chain.invoke({"question": question, "answer": initial_answer})
improved_answer = improvement_chain.invoke({
    "question": question,
    "initial_answer": initial_answer,
    "reflection": reflection
})

# Ergebnisse anzeigen
display(Markdown("# 🔍 Reflective Agent Prozess"))
display(Markdown("## Initiale Antwort:"))
display(Markdown(initial_answer))
display(Markdown("## Selbstreflexion:"))
display(Markdown(reflection))
display(Markdown("## Verbesserte Antwort:"))
display(Markdown(improved_answer))

# Beispielausgabe für verbesserte Antwort:
# Die Hauptursachen des Klimawandels lassen sich in zwei Kategorien einteilen: natürliche und
# menschengemachte Ursachen. Die wissenschaftliche Gemeinschaft ist sich einig, dass die aktuellen
# Veränderungen des Klimas überwiegend auf menschliche Aktivitäten zurückzuführen sind...

## 2.4 Function-Calling



**Kernprinzip:** Standardisierte Schnittstellen für den Aufruf externer Funktionen und Dienste.

**Charakteristika:**
- Direkte Integration: Nahtlose Verbindung zu externen Funktionen
- Standardisierte Schnittstellen: Klare Definitionen für Ein- und Ausgaben
- Spezialisierte Aktionen: Fokus auf spezifische Funktionalitäten

**Reine Function-Calling-Implementierung:**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Prompt für den Function-Calling Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent, der Informationen über das Wetter und andere Fakten liefert."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Function-Calling Agent (fokussiert auf direkten Funktionsaufruf)
function_agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=function_agent,
    tools=tools,
    verbose=True
)

# Typisches Function-Calling Beispiel
location = "Berlin"
response = agent_executor.invoke({
    "input": f"Wie ist das Wetter in {location}? und was gibt es für allgemeine Informationen zu {location}?",
})

# Ergebnis anzeigen
display(Markdown("# 🌡️ Function-Calling Agent Ergebnis"))
display(Markdown(response["output"]))

# Beispielausgabe:
# Das aktuelle Wetter in Berlin ist 8.38°C mit vereinzelten Wolken.
# Berlin ist die Hauptstadt Deutschlands und eine bedeutende kulturelle,
# politische und wirtschaftliche Metropole in Europa.

## 2.5 Fazit

  
Die Entwicklung von KI-Agentenarchitekturen hat in den letzten Jahren eine rasche Evolution durchlaufen:

+ Frühe regelbasierte Agenten: Vor dem Aufkommen moderner LLMs basierten Agenten oft auf vordefinierten Regeln und einfachen Entscheidungsbäumen.
+ Tool-augmentierte Sprachmodelle: Mit der Entwicklung leistungsfähiger LLMs wurden erste Systeme geschaffen, die Sprachmodelle mit externen Tools verbinden.
+ Transparente Reasoning-Architekturen: Architekturen wie Chain-of-Thought und ReAct führten explizite Reasoning-Schritte ein, um die Entscheidungsfindung nachvollziehbarer zu machen.
+ Modulare Systeme: Ansätze wie MRKL ermöglichten die Integration spezialisierter Module für unterschiedliche Aufgabentypen.
+ Autonome Agentenarchitekturen: Neuere Systeme wie AutoGPT und BabyAGI führten zu selbstständigeren Agenten, die eigenständig Ziele verfolgen können.
+ Selbstreflektierende Agenten: Die neueste Generation von Agenten kann ihre eigenen Entscheidungen und Aktionen analysieren und verbessern.

<p><font color='black' size="5">
Gegenüberstellung
</font></p>

| Aspekt | ReAct | MRKL | Reflective Agent | Function-Calling |
|--------|-------|------|------------------|------------------|
| Hauptfokus | Reasoning-Handlungs-Zyklus | Modulare Spezialisierung | Selbstverbesserung | Funktionsintegration |
| Arbeitsweise | Zyklischer Ablauf von Denken und Handeln | Routing zu spezialisierten Modulen | Mehrfache Iteration mit Selbstkritik | Direkte Ausführung externer Funktionen |
| Typischer Anwendungsfall | Mehrstufige Recherche, Problemlösung mit Begründung | Komplexe Fragen, die verschiedene Expertisen erfordern | Content-Erstellung, Code-Review, Fehlersuche | Wetterabfragen, Terminplanung, Datenbankabfragen |
| Flexibilität | Mittel | Hoch | Mittel | Begrenzt auf definierte Funktionen |
| Transparenz | Hoch (durch explizite Gedanken) | Mittel | Sehr hoch | Mittel |
| Komplexitätsbewältigung | Gut für sequentielle Probleme | Gut für diverse Problemtypen | Gut für komplizierte Aufgaben | Gut für definierte Aufgaben |
| Implementation | Prompting-Techniken | Modulare Architektur | Iteration und Feedback | API-Definitionen |

<p><font color='black' size="5">
Kombinationsmöglichkeiten
</font></p>

Die Stärke moderner KI-Agenten liegt oft in der Kombination dieser Architekturen:

- **ReAct + Function-Calling**: Verbindet strukturierte API-Aufrufe mit transparentem Reasoning
- **MRKL + Reflexion**: Modulare Systeme mit Selbstverbesserungsfähigkeit
- **Function-Calling + MRKL**: Standardisierte Schnittstellen für modulare Systeme



# **3 <font color='orange'>|</font> Erweitere Konzepte**
---

Nach der  Darstellung der grundlegenden Architekturen folgen nun erweiterte Konzepte, die auf diesen Architekturen aufbauen oder sie kombinieren.

## 3.1 Einfache Planung



Planung und Zielverfolgung ist kein eigenständiger Architekturtyp, sondern ein Anwendungskonzept, das auf den grundlegenden Architekturen aufbaut.

**Kombination:** Meist basierend auf ReAct oder Reflective Agents

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Chain-of-Thought Prompt erstellen
cot_prompt = PromptTemplate(
    input_variables=["problem"],
    template="""
    Zerlege das folgende Problem in kleinere Teilschritte und löse es Schritt für Schritt:

    Problem: {problem}

    Denke schrittweise:
    1. Welche Teilprobleme müssen gelöst werden?
    2. Wie löse ich jeden Teilschritt?
    3. Wie kombiniere ich die Teillösungen zur Gesamtlösung?
    """
)

# LCEL-Pipeline erstellen und ausführen
planning_chain = (
    {"problem": RunnablePassthrough()}
    | cot_prompt
    | llm
    | StrOutputParser()
)

# Beispielanwendung
complex_problem = """
Berechne die Gesamtkosten für einen Einkauf von 3 Büchern zu je 12,99 €,
2 Notizbüchern zu je 4,50 € und einem Stift für 2,25 €,
wenn auf Bücher 7% Mehrwertsteuer und auf andere Artikel 19% Mehrwertsteuer anfällt.
Verwende für Formeln das Format $ Formel $.
"""

solution = planning_chain.invoke(complex_problem)

# Ergebnis anzeigen
display(Markdown("# 📝 Einfache Planung mit Chain-of-Thought"))
display(Markdown(solution))

# Beispielausgabe:
# Um die Gesamtkosten zu berechnen, muss ich die Kosten für die Bücher und die anderen
# Artikel separat berechnen, die Mehrwertsteuer hinzufügen und dann alles addieren...

## 3.2 Fortgeschrittene Planung



**Kombination:** Oft MRKL + Reflective Agents für hierarchische Planung

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def create_planner_chain():
    """Erstellt eine Chain für die hierarchische Planung."""
    planer_prompt = PromptTemplate.from_template(
        """
        Erstelle einen hierarchischen Plan, um das folgende Ziel zu erreichen:

        Ziel: {objective}

        Teile den Plan in:
        1. Strategische Ziele (hohe Ebene)
        2. Taktische Schritte (mittlere Ebene)
        3. Konkrete Aktionen (detaillierte Ebene)

        Für jede konkrete Aktion gib an, welche Tools oder Ressourcen benötigt werden.
        """
    )
    return ({"objective": RunnablePassthrough()} | planer_prompt | llm | StrOutputParser())

def create_execution_chain():
    """Erstellt eine Chain für die Anpassung des Plans basierend auf Feedback."""
    execution_prompt = PromptTemplate.from_template(
        """
        Hier ist der aktuelle Plan:

        {plan}

        Basierend auf dem folgenden Feedback, passe den Plan an:

        Feedback: {feedback}

        Überarbeiteter Plan:
        """
    )
    return ({"plan": RunnablePassthrough(), "feedback": RunnablePassthrough()}
            | execution_prompt | llm | StrOutputParser())

# Planungsketten erstellen
planner_chain = create_planner_chain()
execution_chain = create_execution_chain()

# Beispiel ausführen
objective = "Organisiere eine zweitägige Konferenz für 100 Teilnehmer zum Thema KI-Ethik"
initial_plan = planner_chain.invoke(objective)

# Feedback geben und Plan anpassen
feedback = "Der Budgetrahmen wurde auf 15.000 € reduziert, und wir müssen die Konferenz auf einen Tag verkürzen."
revised_plan = execution_chain.invoke({"plan": initial_plan, "feedback": feedback})

# Ergebnisse anzeigen
display(Markdown("# 📔 Hierarchische Planung"))
display(Markdown("## Initialer Plan"))
display(Markdown(initial_plan))
display(Markdown("## Überarbeiteter Plan"))
display(Markdown(revised_plan))

# Beispielausgabe für überarbeiteten Plan:
# # Überarbeiteter Plan für eintägige KI-Ethik Konferenz
#
# # Strategische Ziele
# 1. Durchführung einer gehaltvollen, komprimierten eintägigen Konferenz zum Thema KI-Ethik
# 2. Einhaltung des reduzierten Budgetrahmens von 15.000 €
# ...

## 3.3 Zielorientierter Agent



**Kombination:** Function-Calling + ReAct für zielgerichtete Aktionen

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from IPython.display import display, Markdown

# LLM initialisieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt erstellen mit ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent, der Informationen recherchiert. "
                "Nutze die bereitgestellten Tools, um präzise und aktuelle Informationen zu finden."),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Memory für Chat-Verlauf
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# Agent mit OpenAI Functions erstellen
agent = create_openai_functions_agent(llm, tools, prompt)

# Agent-Executor konfigurieren
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# Komplexe Anfrage
complex_goal = """
Finde heraus, wer der aktuelle Bundeskanzler von Deutschland ist,
wann er geboren wurde und erstelle eine kurze Zusammenfassung
seiner politischen Karriere.
"""

# Agent ausführen
result = agent_executor.invoke({"input": complex_goal})

# Ergebnis anzeigen
display(Markdown("# 🔎 Zielorientierter Agent"))
display(Markdown(result["output"]))

# Beispielausgabe:
# Der aktuelle Bundeskanzler von Deutschland ist Olaf Scholz. Er wurde am 14. Juni 1958
# in Osnabrück geboren. Scholz ist Mitglied der SPD und hat vor seinem Amt als Bundeskanzler
# verschiedene politische Positionen inne gehabt...

## 3.4 Kollaborative Analyse



**Kombination:** Spezialisierte Agenten mit verschiedenen Architekturen

In [ ]:
from IPython.display import display, Markdown
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LCEL-Pipelines mit unterschiedlichen Temperaturen und Rollen definieren
research_pipeline = PromptTemplate.from_template(
    """Du bist ein Recherche-Spezialist. Sammle Informationen zum Thema: {topic}"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # MRKL-ähnlich für Informationssuche

critique_pipeline = PromptTemplate.from_template(
    """Du bist ein kritischer Analyst. Bewerte folgende Recherche: {research}"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Reflective-Agent für kritische Analyse

synthesis_pipeline = PromptTemplate.from_template(
    """Du bist ein Informationssynthetisierer. Erstelle eine Zusammenfassung:
    Recherche: {research}
    Kritik: {critique}"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0.3)  # ReAct-ähnlich für Verarbeitung mehrerer Inputs

decision_pipeline = PromptTemplate.from_template(
    """Du bist ein Entscheidungsträger. Basierend auf: {synthesis}
    Beantworte: {question}"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Function-Calling-ähnlich für finale Entscheidung

def collaborative_analysis(topic, question):
    """Kollaborative Analyse mit LCEL-Pipelines."""
    # Prozess ausführen
    research = research_pipeline.invoke({"topic": topic})
    display(Markdown("## 🔍 Recherche abgeschlossen"))

    critique = critique_pipeline.invoke({"research": research})
    display(Markdown("## 🌡️ Kritische Analyse abgeschlossen"))

    synthesis = synthesis_pipeline.invoke({"research": research, "critique": critique})
    display(Markdown("## ✨ Synthese abgeschlossen"))

    decision = decision_pipeline.invoke({"synthesis": synthesis, "question": question})
    display(Markdown("## 📔 Entscheidung getroffen"))

    return {
        "research": research,
        "critique": critique,
        "synthesis": synthesis,
        "decision": decision
    }

# Beispielanwendung
topic = "Künstliche Intelligenz in der Gesundheitsversorgung"
question = "Sollten Krankenhäuser KI-Systeme zur Diagnoseunterstützung einsetzen?"

# Multi-Agenten-Analyse ausführen
display(Markdown("# 👥 Multi-Agenten-System: Kollaborative Analyse"))
results = collaborative_analysis(topic, question)

# Finale Entscheidung anzeigen
display(Markdown(f"# 📑 Finale Entscheidung"))
display(Markdown(f"{results['decision'].content}"))

# Beispielausgabe:
# Basierend auf der vorliegenden Synthese sollten Krankenhäuser KI-Systeme zur
# Diagnoseunterstützung einsetzen, jedoch unter bestimmten Bedingungen und mit
# angemessenen Vorsichtsmaßnahmen...

## 3.5 Agent-Kommunikation



**Kombination:** Agenten mit Function-Calling für standardisierte Kommunikation

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from IPython.display import Markdown, display

class ModernAgent:
    """Moderner Agent für Multi-Agenten-Kommunikation mit LangChain"""
    def __init__(self, name, role, llm, tools=tools):
        self.name = name
        self.role = role
        self.llm = llm
        self.memory = ConversationBufferMemory(return_messages=True)

        # System-Prompt definieren
        system_prompt = f"Du bist {self.name}, ein KI-Agent mit der Rolle: {self.role}"

        # Mindestens ein Tool bereitstellen (wird für alle Agenten benötigt)
        self.tools = tools

        # Prompt-Template erstellen (mit agent_scratchpad für OpenAI Functions Agent)
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        # Agent erstellen
        self.agent = create_openai_functions_agent(llm=self.llm, tools=self.tools, prompt=prompt)

        # Agent-Executor erstellen
        self.agent_executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True)

    def receive_message(self, message, sender):
        """Speichert eine empfangene Nachricht im Gedächtnis."""
        self.memory.chat_memory.add_message(HumanMessage(content=f"{sender}: {message}"))

    def process_and_respond(self, query=None):
        """Verarbeitet den Kontext und generiert eine Antwort mit dem Agent-Executor."""
        # Kontext aus dem Speicher holen
        memory_variables = self.memory.load_memory_variables({})
        memory_messages = memory_variables.get("history", [])
        memory_context = "\n".join([f"{msg.type}: {msg.content}" for msg in memory_messages])

        # Eingabe für den Agent erstellen
        input_text = f"Kontext der bisherigen Konversation:\n{memory_context}\n\n"
        input_text += f"Aufgabe/Frage: {query or 'Reagiere auf die letzte Nachricht.'}"

        # Antwort mit dem Agent-Executor generieren
        response = self.agent_executor.invoke({"input": input_text})

        # Antwort zum Speicher hinzufügen
        self.memory.chat_memory.add_message(AIMessage(content=response["output"]))

        return response["output"]

# Multi-Agenten-System demonstrieren
display(Markdown("# 🤝 Multi-Agenten-System: Kommunikation zwischen Spezialisten"))

# LLM initialisieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Agenten mit verschiedenen Rollen erstellen
researcher = ModernAgent("Forscher", "Sammle und analysiere Informationen zu einem Thema", llm, tools=tools)
critic = ModernAgent("Kritiker", "Bewerte Informationen kritisch und identifiziere mögliche Fehler", llm)
summarizer = ModernAgent("Zusammenfasser", "Erstelle prägnante Zusammenfassungen von Informationen", llm)

# Simulierte Interaktion zwischen Agenten
researcher.receive_message("Recherchiere die Vor- und Nachteile erneuerbarer Energien", "Benutzer")
research_result = researcher.process_and_respond()

critic.receive_message(research_result, "Forscher")
critique = critic.process_and_respond()

summarizer.receive_message(research_result, "Forscher")
summarizer.receive_message(critique, "Kritiker")
final_summary = summarizer.process_and_respond("Erstelle eine ausgewogene Zusammenfassung über erneuerbare Energien")

# Anzeige der Ergebnisse
display(Markdown(f"# 🔍 Forschungsergebnis"))
display(Markdown(f"{research_result}\n"))
display(Markdown(f"# 📣 Kritik"))
display(Markdown(f"{critique}\n"))
display(Markdown(f"# 📓 Endgültige Zusammenfassung"))
display(Markdown(f"{final_summary}"))

# Beispielausgabe:
# Erneuerbare Energien bieten zahlreiche Vorteile wie Nachhaltigkeit, reduzierte Umweltbelastung
# und wirtschaftliche Chancen, stehen jedoch vor Herausforderungen wie Intermittenz,
# Speicherbedarf und anfänglichen Investitionskosten...


# **4 <font color='orange'>|</font> Ethik & Verantwortung**
---

## 4.1 Autonomie und Kontrolle



**Herausforderung der Autonomie:**

- Spannungsfeld zwischen Autonomie und Kontrolle bei KI-Agenten
- Risiken bei zu viel Handlungsfreiheit (Ressourcenverschwendung, unbeabsichtigte Aktionen)
- Notwendigkeit von "Guardrails" und Stopp-Mechanismen

**Technische Lösungsansätze:**

In [ ]:
# Implementierung von Sicherheitsmechanismen
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=15,                # Verhindert Endlosschleifen
    early_stopping_method="force",    # Erzwingt einen Stopp nach max_iterations
    max_execution_time=60,            # Zeitlimit in Sekunden
    handle_parsing_errors=True,       # Robustes Fehlerhandling
    tool_error_handling="continue"    # Bei Tool-Fehlern weitermachen, statt abzubrechen
)

## 4.2 Verantwortung und Haftung



- Wer trägt die Verantwortung für Agenten-Entscheidungen?
- Haftungskette: Entwickler, Betreiber, Nutzer
- Rechtliche Rahmenbedingungen und ihre Lücken
- Dokumentationspflichten bei autonomen Systemen



## 4.3 Verzerrungen und Fairness



**Systematische Verzerrungen in Agenten:**

- Übertragung von Biases aus Trainingsdaten
- Verstärkung durch iterative Entscheidungsprozesse
- Besondere Risiken bei Multi-Agenten-Systemen (Echokammern)

**Implementierung von Fairness-Checks:**

In [ ]:
from IPython.display import display, Markdown

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM für Fairness-Prüfung initialisieren
fairness_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Niedrige Temperatur für objektivere Analyse

# Prompt für die Bias-Prüfung definieren
fairness_prompt = PromptTemplate.from_template("""
Überprüfe folgende Antwort auf mögliche Verzerrungen,
Stereotypen oder unfaire Behandlung bestimmter Gruppen:

Frage: {question}
Antwort: {answer}

Identifiziere jede Form von Bias oder Unfairness:
""")

# Pipeline erstellen
fairness_check = fairness_prompt | fairness_llm | StrOutputParser()

# Beispielfrage und -antwort definieren
question = "Wer eignet sich am besten für Führungspositionen in der Technikbranche?"
initial_answer = "Männer mit technischem Hintergrund eignen sich am besten für Führungspositionen in der Technikbranche, da sie typischerweise analytischer denken und besser mit Stress umgehen können."

# Fairness-Check durchführen
fairness_result = fairness_check.invoke({
    "question": question,
    "answer": initial_answer
})

# Ergebnis ausgeben
display(Markdown("# ⚖️ Fairness-Prüfung"))
display(Markdown(fairness_result))

In [ ]:
# Reflective Agent mit Bias-Prüfung
fairness_prompt = PromptTemplate.from_template("""
Überprüfe folgende Antwort auf mögliche Verzerrungen,
Stereotypen oder unfaire Behandlung bestimmter Gruppen:

Frage: {question}
Antwort: {answer}

Identifiziere jede Form von Bias oder Unfairness:
""")

fairness_check = fairness_prompt | fairness_llm | StrOutputParser()
fairness_result = fairness_check.invoke({
    "question": question,
    "answer": initial_answer
})

## 4.4 Ethische Grundprinzipien



- Transparenz: Nachvollziehbarkeit von Entscheidungen
- Menschliche Kontrolle: Stets die Möglichkeit des menschlichen Eingreifens
- Schadensvermeidung: Priorisierung der Sicherheit
- Datenschutz: Minimierung der Datenzugriffe
- Praktische Umsetzung dieser Prinzipien im Entwicklungsprozess



# **5 <font color='orange'>|</font> Evaluation von Agents**
---

## 5.1 Evaluationsrahmenwerk



**Mehrdimensionales Bewertungssystem:**

- Effektivität: Zielerreichungsgrad
- Effizienz: Ressourcenverbrauch (Zeit, API-Calls, Tokens)
- Robustheit: Verhalten bei unerwarteten Inputs
- Sicherheit: Vermeidung gefährlicher Aktionen

**Implementierung einer Evaluationspipeline:**

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

# Vereinfachte Evaluationskonfiguration ohne TraceAnalyzer
eval_config = RunEvalConfig(
    evaluators=[
        "qa",                          # Qualität der Antworten
        "context_relevance",           # Relevanz verwendeter Informationen
        "helpfulness",                 # Nützlichkeit für den Nutzer
        "correctness",                 # Faktische Korrektheit
    ],
    custom_metrics={
        "completed_in_time": lambda run: run.execution_time < 30,
        "used_appropriate_tools": lambda run: analyze_tool_selection(run)
    }
)

## 5.2 Quantitative Metriken



- **Erfolgsrate:** Anteil erfolgreich gelöster Aufgaben
- **Effizienzmetriken:**
    - Anzahl der Reasoning-Schritte
    - Anzahl der Tool-Aufrufe
    - Token-Verbrauch
    - Ausführungszeit
- **Fehleranalyse:**
    - Typische Fehlermuster
    - Kritische Pfade bei Fehlschlägen



## 5.3 Qualitative Evaluation



- Menschliche Bewertung der Agenten-Outputs
- Think-aloud Protokolle zur Bewertung des Reasoning
- A/B-Tests verschiedener Agentenarchitekturen
- Usability und Nutzererfahrung



## 5.4 Kontinuierliche Verbesserung

# **A <font color='orange'>|</font> Aufgabe**
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

Gegeben ist eine Datei, die eine Reihe von Gleichungen enthält.
Der Dateiname ist GenAI/02 data/gleichungen.txt

**Gleichung:**    
41748459 - 87226336    
92995162 * 46769739    
61530438 * 56074589    
95329602 + 45418854    
412907 + 3731910    
...

Verwenden Sie einen LangChain-Agenten mit einem Tool, um jede dieser Gleichungen zu berechnen, und senden Sie eine Datei ähnlich dieser:

**Ergebnisse:**  
41748459 - 87226336 = 45477877   
92995162 * 46769739 = 4349359455002718   
61530438 * 56074589 = 3450294021839982   
95329602 + 45418854 = 140748456   
412907 + 3731910 = 4144817   
... ...



Mit dieser Aufgabe wenden die Studierenden das Gelernte über Agentenarchitekturen praktisch an. Der Agent verwendet dabei Function-Calling, um:
1. Eine Datei zu lesen (Umgebungsinteraktion)
2. Mathematische Gleichungen zu identifizieren und zu berechnen
3. Die Ergebnisse in einer neuen Datei zu speichern

Dies demonstriert die praktische Anwendung von Tools innerhalb einer Agentenarchitektur für eine reale Aufgabe.



**Feedback-Schleife:**

In [ ]:
# Sammlung von Nutzerfeedback
def collect_feedback(agent_response, user_rating, user_comment):
    feedback_data = {
        "input": agent_response["input"],
        "output": agent_response["output"],
        "rating": user_rating,  # z.B. 1-5 Sterne
        "comment": user_comment,
        "trace": agent_response["intermediate_steps"]
    }
    # Speichern für spätere Analyse und Modellverbesserung
    feedback_db.insert(feedback_data)

    # Kritische Fehler sofort melden
    if user_rating < 2:
        alert_development_team(feedback_data)

    return "Feedback erfolgreich gespeichert"